In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
from loadData import *
from ML_util import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
learning_rate = 0.001
training_epochs = 1000
batch_size = 32

In [6]:
class RBF(nn.Module):
    def __init__(self, in_features, out_features):
        super(RBF, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.centers = nn.Parameter(torch.Tensor(out_features, in_features))
        self.sigma = nn.Parameter(torch.Tensor(out_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.uniform_(self.centers, -1, 1)
        nn.init.constant_(self.sigma, 10)

    def forward(self, x):
        size = (x.size(0), self.out_features, self.in_features)
        x = x.unsqueeze(1).expand(size)
        c = self.centers.unsqueeze(0).expand(size)
        distances = torch.sum((x - c) ** 2, -1)
        return torch.exp(-distances / (2 * self.sigma.unsqueeze(0) ** 2))

In [7]:
class IdentiGazeDataset(Dataset):
    def __init__(self, path, dataType) -> None:
        super().__init__()
        self.loadSelectiveData = LoadResultData(path)
        if dataType == 'train':
            self.data = self.loadSelectiveData.take_df_by_session([1,2,3,4]).reset_index(drop=True)
        elif dataType == 'test':
            self.data = self.loadSelectiveData.take_df_by_session([5]).reset_index(drop=True)

        self.rawgaze = self.loadSelectiveData.take_xy2d(self.data)
        self.eyemovement = self.loadSelectiveData.take_EyeMovement(self.data)
        self.fixation = self.loadSelectiveData.take_Fixation(self.data)
        self.saccade = self.loadSelectiveData.take_Saccade(self.data)
        self.MFCC = self.loadSelectiveData.take_MFCC(self.data)
        self.pupil = self.loadSelectiveData.take_Pupil(self.data)
        self.y = self.loadSelectiveData.take_y(self.data)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        rawgaze = torch.FloatTensor(self.rawgaze.iloc[index]).transpose(0,1)
        eyemovement = torch.FloatTensor(self.eyemovement.iloc[index])
        fixation = torch.FloatTensor(self.fixation.iloc[index])
        saccade = torch.FloatTensor(self.saccade.iloc[index])
        mfcc = torch.FloatTensor(self.MFCC.iloc[index])
        pupil = torch.FloatTensor(self.pupil.iloc[index])
        label = self.y.iloc[index]
        return {'rawgaze':rawgaze, 'eyemovement':eyemovement, 'fixation':fixation, 'saccade': saccade, 
                'mfcc': mfcc, 'pupil': pupil, 'y': label}

path = 'C:\\Users\\scilab\\IdentiGaze\\data\\DayDifference\\Similar_All.csv'
trainDataset = IdentiGazeDataset(path, 'train')
testDataset = IdentiGazeDataset(path, 'test')


In [8]:
class RBFN(nn.Module):
    def __init__(self, in_features, out_features, rbf_neurons):
        super(RBFN, self).__init__()
        self.rbf = RBF(in_features, rbf_neurons)
        self.linear = nn.Linear(rbf_neurons, out_features)

    def forward(self, x):
        x = self.rbf(x)
        x = self.linear(x)
        return x

In [4]:
class IdentiRBFNet(nn.Module):
    def __init__(self):
        super(IdentiRBFNet, self).__init__()
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, 128)
        # Define the output layer
        self.output = nn.Linear(128, 34)

        self.EyeMovementNet = nn.Sequential(
            RBF(9, 64),
            nn.Linear(64, 34)
        )

        self.FixationNet = nn.Sequential(
            RBF(10, 64),
            nn.Linear(64, 34)
        )

        self.SaccadeNet = nn.Sequential(
            RBF(17, 64),
            nn.Linear(64, 34)
        )

        self.MFCCNet = nn.Sequential(
            RBF(12, 64),
            nn.Linear(64, 34)
        )

        self.PupilNet = nn.Sequential(
            RBF(12, 64),
            nn.Linear(64, 34)
        )


    # def forward(self, rawgaze, eyemovement, fixation, saccade, mfcc, pupil):
    def forward(self, eyemovement, fixation, saccade, mfcc, pupil):
        # Apply the gated convolutions
        # rawgaze = self.RawGazeNet(rawgaze)
        # rawgaze = rawgaze.view(rawgaze.size(0), -1)
        # rawgaze = F.relu(self.fc1(rawgaze))
        # rawgaze = self.dropout(rawgaze)
        # rawgaze = F.relu(self.fc2(rawgaze))
        # rawgaze = self.output(rawgaze)

        eyemovement = eyemovement.view(eyemovement.size(0), -1)
        eyemovement = self.EyeMovementNet(eyemovement)
        fixation = self.FixationNet(fixation)
        saccade = self.SaccadeNet(saccade)
        mfcc = self.MFCCNet(mfcc)
        pupil = self.PupilNet(pupil)

        # out = rawgaze + eyemovement + fixation + saccade + mfcc + pupil
        out = eyemovement + fixation + saccade + mfcc + pupil
        return out


In [18]:
class IdentiDenseNet(nn.Module):
    def __init__(self):
        super(IdentiDenseNet, self).__init__()
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, 128)
        # Define the output layer
        self.output = nn.Linear(128, 34)

        self.EyeMovementNet = nn.Sequential(
            nn.Linear(9, 64),
            nn.ReLU(),
            nn.Linear(64, 34)
        )

        self.FixationNet = nn.Sequential(
            nn.Linear(10, 64),
            nn.ReLU(),
            nn.Linear(64, 34)
        )

        self.SaccadeNet = nn.Sequential(
            nn.Linear(17, 64),
            nn.ReLU(),
            nn.Linear(64, 34)
        )

        self.MFCCNet = nn.Sequential(
            nn.Linear(12, 64),
            nn.ReLU(),
            nn.Linear(64, 34)
        )

        self.PupilNet = nn.Sequential(
            nn.Linear(12, 64),
            nn.ReLU(),
            nn.Linear(64, 34)
        )


    # def forward(self, rawgaze, eyemovement, fixation, saccade, mfcc, pupil):
    def forward(self, eyemovement, fixation, saccade, mfcc, pupil):
        # Apply the gated convolutions
        # rawgaze = self.RawGazeNet(rawgaze)
        # rawgaze = rawgaze.view(rawgaze.size(0), -1)
        # rawgaze = F.relu(self.fc1(rawgaze))
        # rawgaze = self.dropout(rawgaze)
        # rawgaze = F.relu(self.fc2(rawgaze))
        # rawgaze = self.output(rawgaze)

        eyemovement = eyemovement.view(eyemovement.size(0), -1)
        eyemovement = self.EyeMovementNet(eyemovement)
        fixation = self.FixationNet(fixation)
        saccade = self.SaccadeNet(saccade)
        mfcc = self.MFCCNet(mfcc)
        pupil = self.PupilNet(pupil)

        # out = rawgaze + eyemovement + fixation + saccade + mfcc + pupil
        out = eyemovement + fixation + saccade + mfcc + pupil
        return out


In [26]:
class CausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dilation=1):
        super(CausalConv1d, self).__init__()
        self.padding = (kernel_size - 1) * dilation
        self.conv1d = nn.Conv1d(in_channels, out_channels, kernel_size, stride=stride, padding=self.padding, dilation=dilation)

    def forward(self, x):
        x = self.conv1d(x)
        return x[:, :, :-self.padding]  # Remove the padding on the right

class GatedActivationUnit(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dilation=1):
        super(GatedActivationUnit, self).__init__()
        self.conv = CausalConv1d(in_channels, out_channels * 2, kernel_size, stride, dilation)

    def forward(self, x):
        x = self.conv(x)
        out, gate = x.chunk(2, dim=1)  # Split the tensor along the channel dimension
        return torch.tanh(out) * torch.sigmoid(gate)  # Gated activation
    

class TotalRBFNet(nn.Module):
    def __init__(self):
        super(TotalRBFNet, self).__init__()
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, 128)
        # Define the output layer
        self.output = nn.Linear(128, 34)


        self.RawGazeNet = nn.Sequential(
            # Input Size: (84,2)
            CausalConv1d(2, 4, kernel_size=7, stride=1, dilation=2),
            nn.MaxPool1d(kernel_size=3, stride=3),
            CausalConv1d(4, 5, kernel_size=5, stride=1, dilation=4),
            CausalConv1d(5, 16, kernel_size=3, stride=1, dilation=5),
            CausalConv1d(16, 32, kernel_size=3, stride=1, dilation=16),
            nn.AdaptiveMaxPool1d(1),  # Global Max Pooling
        )


        self.EyeMovementNet = nn.Sequential(
            RBF(9, 64),
            nn.Linear(64, 34)
        )

        self.FixationNet = nn.Sequential(
            RBF(10, 64),
            nn.Linear(64, 34)
        )

        self.SaccadeNet = nn.Sequential(
            RBF(17, 64),
            nn.Linear(64, 34)
        )

        self.MFCCNet = nn.Sequential(
            RBF(12, 64),
            nn.Linear(64, 34)
        )

        self.PupilNet = nn.Sequential(
            RBF(12, 64),
            nn.Linear(64, 34)
        )


    def forward(self, rawgaze, eyemovement, fixation, saccade, mfcc, pupil):
        # Apply the gated convolutions
        rawgaze = self.RawGazeNet(rawgaze)
        rawgaze = rawgaze.view(rawgaze.size(0), -1)
        rawgaze = F.relu(self.fc1(rawgaze))
        rawgaze = self.dropout(rawgaze)
        rawgaze = F.relu(self.fc2(rawgaze))
        rawgaze = self.output(rawgaze)

        eyemovement = eyemovement.view(eyemovement.size(0), -1)
        eyemovement = self.EyeMovementNet(eyemovement)
        fixation = self.FixationNet(fixation)
        saccade = self.SaccadeNet(saccade)
        mfcc = self.MFCCNet(mfcc)
        pupil = self.PupilNet(pupil)

        out = rawgaze + eyemovement + fixation + saccade + mfcc + pupil
        return out


In [27]:
modelName = 'IdRBF'

writer = SummaryWriter(f'model/{modelName}/runs')
# idModel = IdentiRBFNet()
# idModel = IdentiDenseNet()
idModel = TotalRBFNet()
# output = idModel(torch.randn(1,9).to(device),torch.randn(1,10).to(device),torch.randn(1,17).to(device),torch.randn(1,12).to(device),torch.randn(1,12).to(device))
output = idModel(torch.randn(1,2,84).to(device), torch.randn(1,9).to(device),torch.randn(1,10).to(device),torch.randn(1,17).to(device),torch.randn(1,12).to(device),torch.randn(1,12).to(device))

# print(len(myIdentiGaze))
print(output.shape)

torch.Size([1, 34])


In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(idModel.parameters(), lr=0.01)

In [29]:
train_size = len(trainDataset)
test_size = len(testDataset)
train_loader = DataLoader(trainDataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testDataset, batch_size=batch_size, shuffle=True)

In [31]:
for epoch in range(training_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(train_loader):
        rawgaze = data['rawgaze'].to(device).type(dtype=torch.float32)
        eyemovement = data['eyemovement'].to(device).type(dtype=torch.float32)
        fixation = data['fixation'].to(device).type(dtype=torch.float32)
        saccade = data['saccade'].to(device).type(dtype=torch.float32)
        mfcc = data['mfcc'].to(device).type(dtype=torch.float32)
        pupil = data['pupil'].to(device).type(dtype=torch.float32)
        y = data['y'].to(device)
        
        optimizer.zero_grad()
        # outputs = idModel(eyemovement, fixation, saccade, mfcc, pupil)
        outputs = idModel(rawgaze, eyemovement, fixation, saccade, mfcc, pupil)
        loss = criterion(outputs, y)
        _, predicted = torch.max(outputs.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    Accuracy = 100 * correct // total
    writer.add_scalar('Acc/train', Accuracy, epoch+1)
    writer.add_scalar('Loss/train', running_loss, epoch+1)
    running_loss = 0.0

    
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in test_loader:
            rawgaze = data['rawgaze'].to(device).type(dtype=torch.float32)
            eyemovement = data['eyemovement'].to(device).type(dtype=torch.float32)
            fixation = data['fixation'].to(device).type(dtype=torch.float32)
            saccade = data['saccade'].to(device).type(dtype=torch.float32)
            mfcc = data['mfcc'].to(device).type(dtype=torch.float32)
            pupil = data['pupil'].to(device).type(dtype=torch.float32)
            y = data['y'].to(device)
            # calculate outputs by running images through the network
            outputs = idModel(rawgaze, eyemovement, fixation, saccade, mfcc, pupil)
            # outputs = idModel(eyemovement, fixation, saccade, mfcc, pupil)
            test_loss = criterion(outputs, y)
            writer.add_scalar('Loss/test', test_loss, epoch+1)

            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
        Accuracy = 100 * correct // total
        print(f'Accuracy of the network on the 10000 test images: {Accuracy} %')
        writer.add_scalar('Acc/test', Accuracy, epoch+1)
        torch.save(idModel.state_dict(), f'model/{modelName}/{epoch}_{100 * correct // total}.pth')


C:\Users\scilab\AppData\Local\Temp\ipykernel_7352\474243876.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  eyemovement = torch.FloatTensor(self.eyemovement.iloc[index])
C:\Users\scilab\AppData\Local\Temp\ipykernel_7352\474243876.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fixation = torch.FloatTensor(self.fixation.iloc[index])
C:\Users\scilab\AppData\Local\Temp\ipykernel_7352\474243876.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, u

Accuracy of the network on the 10000 test images: 15 %
Accuracy of the network on the 10000 test images: 13 %
Accuracy of the network on the 10000 test images: 9 %
Accuracy of the network on the 10000 test images: 11 %
Accuracy of the network on the 10000 test images: 12 %
Accuracy of the network on the 10000 test images: 13 %
Accuracy of the network on the 10000 test images: 15 %
Accuracy of the network on the 10000 test images: 15 %
Accuracy of the network on the 10000 test images: 17 %
Accuracy of the network on the 10000 test images: 22 %
Accuracy of the network on the 10000 test images: 22 %
Accuracy of the network on the 10000 test images: 21 %
Accuracy of the network on the 10000 test images: 24 %
Accuracy of the network on the 10000 test images: 21 %
Accuracy of the network on the 10000 test images: 25 %
Accuracy of the network on the 10000 test images: 22 %
Accuracy of the network on the 10000 test images: 23 %
Accuracy of the network on the 10000 test images: 25 %
Accuracy of

KeyboardInterrupt: 